In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from itertools import product

import seaborn as sns
# from run_exp import run_exp, run_exp_with_trans
import os
import re
from collections import Counter
from scipy.stats import chisquare
import matplotlib.pyplot as plt

from icl.dyck.dyck import DyckPathTask, DyckBayes
from icl.dyck.dyck_config import *
from icl.dyck.dyck_utils import *
from icl.dyck.dyck_task_vec import compute_hiddens_dyck
from icl.figures.task_vec_viz import *
from icl.models import Transformer
import icl.utils.notebook_utils as nu
from icl.utils.train import *
from icl.figures.head_view import *
from icl.linear.linear_utils import estimate_lambda_with_r2

torch.set_printoptions(precision=3, sci_mode=False)

%load_ext autoreload
%autoreload 2

In [17]:
config = get_config_dyck()
config.task.n_minor_tasks = 1
model = Transformer(config)
model = model.to(config.device)

_ = train_model_with_plot(model, config, show=False)

Experiment directory:  ..\results\dyck\train_9a0fb4e67868ea3fe1f6c1395ad84908
train_9a0fb4e67868ea3fe1f6c1395ad84908 already completed


In [18]:
import copy 

exp_name = "train_9a0fb4e67868ea3fe1f6c1395ad84908"
model, sampler, config = nu.load_everything("dyck", exp_name)

def get_new_sampler(exp_name, n_tasks=61, n_ood=40):
    model, sampler, config = nu.load_everything("dyck", exp_name)
    sampler_clone0 = copy.deepcopy(sampler)

    orig = sampler_clone0.minor_task_pool
    k_ood = max(n_ood, n_tasks - sampler_clone0.n_minor_tasks)
    sampler_clone0.n_minor_tasks = n_tasks
    k_minor = n_tasks - k_ood  # how many "old" rows we want after the OOD block

    ood = sampler_clone0._random_dyck_path(k_ood)
    if orig is not None:
        sampler_clone0.minor_task_pool = torch.cat([orig[:k_minor], ood], dim=0)
    else:
        sampler_clone0.minor_task_pool = ood

    return sampler_clone0, k_minor
    
sampler_clone0, k_minor = get_new_sampler(exp_name)


In [20]:
mask = sample_binary_mask(config)

hiddens = compute_hiddens_dyck(config,
                    model,
                    sampler = sampler_clone0,
                    dyck_mask = mask,
                    layer_index = 4,
                    )
plot_task_vector_modes(hiddens[:4])

In [16]:
a = 0
b = 9

def plot_hidden_proj(hiddens, mask):
    K, T, B, D = hiddens.shape
    global_mean = hiddens[:3].mean(dim=(0,2)) # (T, D)
    task_vecs_over_all_time = hiddens.mean(dim=2) - global_mean.unsqueeze(dim=0) # (K, T, D)
    final_task_vecs = task_vecs_over_all_time[:3, -1]
    lambdas, r2_scores, task_norms, ortho_norms = estimate_lambda_with_r2(final_task_vecs, task_vecs_over_all_time)
    fig = project_with_r2_size(task_vecs_over_all_time, final_task_vecs, r2_scores, lambdas=lambdas, mask=mask, n_minors=k_minor)
    fig.show()

idx = torch.nonzero(mask == 1, as_tuple=True)[0]
plot_hidden_proj(hiddens[:, idx[a].item():idx[b].item()+1], mask[idx[a].item():idx[b].item()+1])

In [21]:
idx = torch.nonzero(mask == 1, as_tuple=True)[0]
plot_hidden_proj(hiddens[:, idx[a].item():idx[b].item()+1], mask[idx[a].item():idx[b].item()+1])